<a href="https://colab.research.google.com/github/profteachkids/CHE2064_Spring2023/blob/main/IsopropanolSynthesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/che4.ipynb -O che4.ipynb
!python -m pip install importnb --quiet
from importnb import Notebook
with Notebook():
    from che4 import Range, Comp, CompArray, RangeArray, DotDict, d2nt, Props

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 1.8 MB/s eta 0:00:00


In [6]:
import numpy as np
from scipy.optimize import root
import jax
import jax.numpy as jnp

In [7]:
#rx1 C3H6 + H2O -> C3H7OH
#rx2 2C3H7OH -> (C3H7)O(C3H7) + H2O
#1. C3H6, 2. (C3H7)O(C3H7), 3. C3H7OH, 4. H2O

e=DotDict()
e.stoic_rx1 = np.array([-1., 0., 1., -1.])
e.stoic_rx2 = np.array([0., 1., -2, 1.])

e.alkeneF = 100.
e.alkeney = np.array([1., 0., 0., 0.])
e.waterF = 95.
e.waterx = np.array([0., 0., 0., 1.])

e.etherx = np.array([0.001, 0.9, 0.09, 0.01])
e.alcoholx = np.array([0.001, 0.01, 0.94, 0.049])

e.etherP = Range(5., 0., e.alkeneF/2)
e.alcoholP = Range(50., 0., e.alkeneF)

e.ext_rx1 = Range(e.alkeneF/2, 0., e.alkeneF)
e.ext_rx2 = Range(e.alkeneF/4, 0., e.alkeneF/2)

In [8]:
x0, x2nt, wrap, x2unk, const=d2nt(e)

In [9]:
def eqs(d):
    feed = d.alkeneF*d.alkeney + d.waterF*d.waterx
    rxns = d.ext_rx1*d.stoic_rx1 + d.ext_rx2*d.stoic_rx2
    prod =  d.etherP*d.etherx + d.alcoholP*d.alcoholx
    return feed + rxns - prod

In [10]:
wrapped_eqs=wrap(eqs)

In [11]:
x=root(wrapped_eqs, x0).x 

In [12]:
x2unk(x)._asdict()

{'etherP': Array(9.2715403, dtype=float64),
 'alcoholP': Array(85.81427397, dtype=float64),
 'ext_rx1': Array(99.90491419, dtype=float64),
 'ext_rx2': Array(9.20252901, dtype=float64)}

In [13]:
const._asdict()

{'stoic_rx1': Array([-1.,  0.,  1., -1.], dtype=float64),
 'stoic_rx2': Array([ 0.,  1., -2.,  1.], dtype=float64),
 'alkeneF': Array(100., dtype=float64),
 'alkeney': Array([1., 0., 0., 0.], dtype=float64),
 'waterF': Array(95., dtype=float64),
 'waterx': Array([0., 0., 0., 1.], dtype=float64),
 'etherx': Array([0.001, 0.9  , 0.09 , 0.01 ], dtype=float64),
 'alcoholx': Array([0.001, 0.01 , 0.94 , 0.049], dtype=float64)}

In [19]:
def eqs2(d):
    reactor_feed=d.alkeneF*d.alkeney + d.waterF*d.waterx + d.D1B*d.D1Bx + d.decanterW*d.decanterWx
    rxns = d.ext_rx1*d.stoic_rx1 + d.ext_rx2*d.stoic_rx2
    reactor_prod = d.D1F*d.D1Fx

    WARFR_CONSTRAINT = (d.waterF + d.D1B + d.decanterW)/d.alkeneF - d.WARFR

    REACTOR_BALANCE = reactor_feed + rxns - reactor_prod

    D1_BALANCE = d.D1F*d.D1Fx - d.D1B*d.D1Bx - d.D1D*d.D1Dy
    D1_VLE = d.D1Dy/d.D1Bx - d.D1K

    AZ_BALANCE = d.D1D*d.D1Dy -  d.decanterW*d.decanterWx -  d.etherP*d.etherx - d.alcoholP*d.alcoholx

    return jnp.concatenate([REACTOR_BALANCE, D1_BALANCE, D1_VLE, AZ_BALANCE, jnp.atleast_1d(WARFR_CONSTRAINT)])

In [23]:
e=DotDict()
e.stoic_rx1 = np.array([-1., 0., 1., -1.])
e.stoic_rx2 = np.array([0., 1., -2, 1.])

e.alkeneF = 100.
e.alkeney = np.array([1., 0., 0., 0.])
e.waterF = 95.
e.waterx = np.array([0., 0., 0., 1.])

e.etherx = np.array([0.001, 0.9, 0.09, 0.01])
e.alcoholx = np.array([0.001, 0.01, 0.94, 0.049])

e.WARFR = 12.        #Water-Alkene Reactor Feed Ratio

e.D1F = Range(e.WARFR*e.alkeneF, 0., 2.* e.WARFR*e.alkeneF)
e.D1Fx = Comp([ 0.01 ,0.03, 0.06,0.9])

e.D1D = Range(e.WARFR*e.alkeneF/2, 0., 2.* e.WARFR*e.alkeneF)
e.D1Dy = Comp([0.01,0.33, 0.33, 0.33])

e.D1B = Range(e.WARFR*e.alkeneF/2, 0., 2.* e.WARFR*e.alkeneF)
e.D1Bx = Comp([0.0001,0.1,0.1,0.8])

e.decanterW = Range(e.WARFR*e.alkeneF, 0., 2.* e.WARFR*e.alkeneF)
e.decanterWx = np.array([ 0.00001 , 0.002, 0.018, 0.98])

e.etherP = Range(5., 0., e.alkeneF/2)
e.alcoholP = Range(50., 0., e.alkeneF)

e.D1K=np.array([100, 10., 5., 0.2])

e.ext_rx1 = Range(e.alkeneF/2, 0., e.alkeneF)
e.ext_rx2 = Range(e.alkeneF/4, 0., e.alkeneF/2)

In [24]:
x0, x2nt, wrap, x2unk, const= d2nt(e)

In [25]:
wrapped_eqs2=wrap(eqs2)

In [27]:
sol=root(wrapped_eqs2, x0, jac=jax.jacobian(wrapped_eqs2))
print(sol)

 message: The solution converged.
 success: True
  status: 1
     fun: [-1.334e-09  2.262e-11 ...  6.573e-12 -1.776e-15]
       x: [ 1.587e-04 -9.099e+00 ...  6.957e+00 -1.489e+00]
    nfev: 39
    njev: 3
    fjac: [[-7.027e-05 -1.345e-02 ...  3.083e-11 -4.086e-14]
           [ 7.057e-01  1.877e-05 ...  2.778e-05 -1.793e-08]
           ...
           [-4.608e-01  3.269e-03 ... -4.069e-01 -1.397e-05]
           [-4.180e-01 -1.928e-03 ...  3.977e-01 -1.890e-10]]
       r: [ 6.848e+02 -1.163e-01 ... -3.362e+00  3.276e+00]
     qtf: [ 1.377e-07  2.053e-07 ... -2.366e-08 -4.928e-13]


In [30]:
x2unk(sol.x)._asdict()

{'D1F': Array(1200.09523025, dtype=float64),
 'D1Fx': Array([8.72524480e-05, 1.53505215e-02, 2.03781986e-01, 7.80780241e-01],      dtype=float64),
 'D1D': Array(109.53910025, dtype=float64),
 'D1Dy': Array([0.00086937, 0.08427508, 0.74639873, 0.16845681], dtype=float64),
 'D1B': Array(1090.55613001, dtype=float64),
 'D1Bx': Array([8.69372240e-06, 8.42750829e-03, 1.49279747e-01, 8.42284051e-01],      dtype=float64),
 'decanterW': Array(14.44386999, dtype=float64),
 'etherP': Array(9.2715403, dtype=float64),
 'alcoholP': Array(85.81427397, dtype=float64),
 'ext_rx1': Array(99.90491419, dtype=float64),
 'ext_rx2': Array(9.20252901, dtype=float64)}

In [30]:
def f(x):
    return np.sin(x)

In [35]:
def derivative(func, h=1e-6):
    def func_prime(x):
        return (func(x+h)-func(x))/h

    return func_prime


In [40]:
fprime = derivative(f, h=1e-8)
fprime(0.5)

0.8775825621754052

In [34]:
np.cos(0.5)

0.8775825618903728

In [1]:
import jax.numpy as jnp
import jax
from jax.config import config
config.update("jax_enable_x64", True)

In [2]:
def f_jax(x):
    return jnp.sin(x)

In [3]:
f_jax_prime=jax.grad(f_jax)

In [4]:
f_jax_prime(0.5)

Array(0.87758256, dtype=float64, weak_type=True)